In [1]:
import tensorflow as tf
from tensorflow import keras

In [3]:
#house dataset
from sklearn.datasets import fetch_california_housing
house = fetch_california_housing()
X = house.data
y = house.target

#rescale
from sklearn.preprocessing import StandardScaler
X_scale = StandardScaler().fit(X).transform(X)

#split data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_scale, y,
                                              test_size=0.2, random_state=0)
print(X_train.shape, X_test.shape)

(16512, 8) (4128, 8)


In [4]:
input = keras.layers.Input(shape=8)
hidden1 = keras.layers.Dense(30,activation='relu')(input)
hidden2 = keras.layers.Dense(30,activation='relu')(hidden1)
concat= keras.layers.Concatenate()([input, hidden2])
output=keras.layers.Dense(1)(concat)
model = keras.models.Model(inputs=[input], outputs=[output])

In [5]:
model.compile(loss='mse',optimizer='sgd')

In [7]:
model.fit(X_train, y_train,epochs=30,validation_split=0.1)

Epoch 1/30
465/465 [==============================] - 1s 2ms/step - loss: 0.3698 - val_loss: 0.3167
Epoch 2/30
465/465 [==============================] - 1s 1ms/step - loss: 0.3659 - val_loss: 0.3161
Epoch 3/30
465/465 [==============================] - 1s 2ms/step - loss: 0.3602 - val_loss: 0.3765
Epoch 4/30
465/465 [==============================] - 1s 2ms/step - loss: 0.3611 - val_loss: 0.3045
Epoch 5/30
465/465 [==============================] - 1s 2ms/step - loss: 0.3559 - val_loss: 0.3023
Epoch 6/30
465/465 [==============================] - 1s 2ms/step - loss: 0.3501 - val_loss: 0.2987
Epoch 7/30
465/465 [==============================] - 1s 1ms/step - loss: 0.3427 - val_loss: 0.2955
Epoch 8/30
465/465 [==============================] - 1s 1ms/step - loss: 0.3448 - val_loss: 0.3043
Epoch 9/30
465/465 [==============================] - 1s 1ms/step - loss: 0.3445 - val_loss: 0.2930
Epoch 10/30
465/465 [==============================] - 1s 1ms/step - loss: 0.3455 - val_loss: 0.3221

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 30)           270         ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 30)           930         ['dense[0][0]']                  
                                                                                                  
 concatenate (Concatenate)      (None, 38)           0           ['input_1[0][0]',                
                                                                  'dense_1[0][0]']            

In [9]:
X_new = X_test[:10]
y_hat=model.predict(X_new)
print(y_hat)
print(y_test[:10])

1/1 [==============================] - 0s 97ms/step
[[1.5670383 ]
 [2.488556  ]
 [1.5042796 ]
 [0.79375416]
 [3.8762136 ]
 [2.4983017 ]
 [2.734212  ]
 [3.6968474 ]
 [2.9463792 ]
 [2.4785085 ]]
[1.369 2.413 2.007 0.725 4.6   1.2   2.47  3.369 3.397 2.656]


In [10]:
import tensorflow as tf
from tensorflow import keras

In [11]:
class WideDeep(keras.models.Model):
    def __init__(self,units=30,activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.hidden1=keras.layers.Dense(units,activation=activation)
        self.hidden2=keras.layers.Dense(units, activation=activation)
        self.main_output=keras.layers.Dense(1)
        self.aux_output=keras.layers.Dense(1)
        
    def call(self,inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2= self.hidden2(hidden1)
        concat = keras.layers.Concatenate()([input_A,hidden2])
        main_output= self.main_output(concat)
        aux_output=self.aux_output(hidden2)
        return main_output, aux_output
        

In [12]:
model = WideDeep()
model.compile(loss='mse',optimizer='sgd')

In [21]:
#X_train
X_train_A,X_train_B= X_train[:,:6:],X_train[:,-5]
print(X_train_A.shape,X_train_B.shape)

(16512, 6) (16512,)
